# Puesta en producción
En este caso pondemos el modelo en producción para poder consumirlo. Usaremos un checkpoint generado anteriormente para ello.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Librerías estandar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from starlette.requests import Request

from pprint import pprint
from progressbar import ProgressBar
from collections import OrderedDict
import tree  # dm_tree


# librerias para RL
import ray
from ray import tune
from ray import serve

from ray.rllib.agents.marwil import BCTrainer, BCConfig # derivado del MARWIL
from ray.rllib.agents.marwil import MARWILTrainer, MARWILConfig
from ray.rllib.offline.estimators import (
    ImportanceSampling,
    WeightedImportanceSampling,
    DirectMethod,
    DoublyRobust,
)
## environment
from ray.rllib.examples.env.recommender_system_envs_with_recsim import LongTermSatisfactionRecSimEnv, InterestEvolutionRecSimEnv



C:\Users\Paul\AppData\Roaming\Python\Python39\site-packages\flatbuffers\compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
C:\Users\Paul\AppData\Roaming\Python\Python39\site-packages\keras\utils\image_utils.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
C:\Users\Paul\AppData\Roaming\Python\Python39\site-packages\keras\utils\image_utils.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
C:\Users\Paul\AppData\Roaming\Python\Python39\site-packages\keras\utils\image_utils.py:38: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
C:\Users\Paul\AppData\Roa

In [3]:
# cargamos las funciones que hemos guardado dentro de la carpeta src
# nos permite compartir funciones en diferentes notebooks
import sys
sys.path.append('../src')
from models.rlmodels import run_random_walk_episode, run_rlmodel_episode, run_simulation, run_one_single_episode
from visualization.visualize import plot_rewards_timestamp
from data.env import LTSWithStrongerDissatisfactionEffect 
from utils.docs import get_json_files

In [4]:
checkpoint_file = '../models/offline_rl/interest_evolution/BCTrainer\checkpoint_000117'
OUTPUT_DIR = "C:\\Users\\Paul\\ray_results\\DQN_2022-09-01_17-40-13\\DQN_env_ie_5e8d7_00000_0_2022-09-01_17-40-13\\data"

SEED = 0

## configuración del entorno: 20 posibles candidatos del que seleccionaremos 2
# mantendremos esta configuración para los demás agentes también
NUM_CANDIDATES = 20
SLATE_SIZE = 2
REGISTER_ENV_NAME = "env_ie"


<>:1: DeprecationWarning: invalid escape sequence \c


In [5]:
# cargamos los datos  ofline del modelo para poder predecir con el modelo
json_output_file = get_json_files(OUTPUT_DIR)[-1]

In [6]:


offline_rl_env = LongTermSatisfactionRecSimEnv({
    "num_candidates": NUM_CANDIDATES,
    "slate_size": SLATE_SIZE,
    "wrap_for_bandits": True,  
    "convert_to_discrete_action_space": True,
})

#Configuramos el entrenador BC
offline_rl_config = {
    "input": [json_output_file],
    "actions_in_input_normalized": True,
    # Para modelos online, esto se configura a "sampler" por defecto.

    # como no tenemos un entorno y el espacio de observaciones y acciones no está definido en el archivo JSON,
    # lo especificamos manualmente
    "env": None,  # defecto
    "observation_space": offline_rl_env.observation_space,
    "action_space": offline_rl_env.action_space,

    # estimaciones OPE 
    "input_evaluation": ["is", "wis"],
}

config_bc = (
    BCConfig()
    .environment(env=None)
    .framework("torch")
    .offline_data(input_=[json_output_file])
    .evaluation(
        evaluation_interval=1,
        evaluation_duration=10,
        evaluation_num_workers=1,
        evaluation_duration_unit="episodes",
        evaluation_config={"input": [json_output_file]},
        off_policy_estimation_methods={
            "is": {"type": ImportanceSampling},
            "wis": {"type": WeightedImportanceSampling},
           
            },
        #},
    )
)
# hemos puesto en env a None por lo que tenemos que darle esta info al modelo offline
config_bc.observation_space = offline_rl_env.observation_space 
config_bc.action_space = offline_rl_env.action_space


In [7]:
bc_trainer = config_bc.build()
bc_trainer.restore(checkpoint_file)
print(f"Cargamos el Trainer en la iteración={bc_trainer.iteration}")

2022-09-05 14:21:26,684	WARNING deprecation.py:47 -- DeprecationWarning: `config['multiagent']['replay_mode']` has been deprecated. config['replay_buffer_config']['replay_mode'] This will raise an error in the future!
C:\Users\Paul\AppData\Roaming\Python\Python39\site-packages\ray\_private\ray_option_utils.py:266: DeprecationWarning: Setting 'object_store_memory' for actors is deprecated since it doesn't actually reserve the required object store memory. Use object spilling that's enabled by default (https://docs.ray.io/en/releases-2.0.0/ray-core/objects/object-spilling.html) instead to bypass the object store memory size limitation.
  warnings.warn(
2022-09-05 14:21:29,909	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2022-09-05 14:21:32,750	WARNING deprecation.py:47 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
2022-09-05 14:21:32,751	WARNING deprecation.py:47 -- DeprecationWarning: 

Before restoring: Trainer is at iteration=0
After restoring: Trainer is at iteration=117


In [8]:
serve.start()

@serve.deployment(route_prefix="/long-term-satisfaction")
class ServeModel:
    def __init__(self, checkpoint_path) -> None:
        self.trainer = config_bc.build()
        self.trainer.restore(checkpoint_path)

    async def __call__(self, request: Request):
        json_input = await request.json()
        obs = json_input["observation"]
        # Translate obs back to np.arrays.
        np_obs = OrderedDict(tree.map_structure(lambda s: np.array(s) if isinstance(s, list) else s, obs))
        action = self.trainer.compute_single_action(np_obs, explore=False)
        return {"action": action}


ServeModel.deploy(checkpoint_file)

(ServeController pid=27152) INFO 2022-09-05 14:21:40,478 controller 27152 http_state.py:129 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:PyKHYc:SERVE_PROXY_ACTOR-74a99897c07e4fe3d3aa2682588827412cfd89a8b216140686d4782f' on node '74a99897c07e4fe3d3aa2682588827412cfd89a8b216140686d4782f' listening on '127.0.0.1:8000'
(HTTPProxyActor pid=22640) INFO:     Started server process [22640]
(ServeController pid=27152) INFO 2022-09-05 14:21:42,417 controller 27152 deployment_state.py:1232 - Adding 1 replicas to deployment 'ServeModel'.
(ServeReplica:ServeModel pid=7532) C:\Users\Paul\AppData\Roaming\Python\Python39\site-packages\flatbuffers\compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
(ServeReplica:ServeModel pid=7532)   import imp
(ServeReplica:ServeModel pid=7532) C:\Users\Paul\AppData\Roaming\Python\Python39\site-packages\keras\utils\image_utils.py:36: DeprecationWarning: NEAREST is deprec

In [9]:
# Request 5 actions of an episode from served policy.
obs = offline_rl_env.reset()

for _ in range(5):
    # Convert numpy arrays to lists (needed for transfer).
    obs = tree.map_structure(lambda s: s.tolist() if isinstance(s, np.ndarray) else s, obs)

    print(f"-> Sending observation {obs}")
    resp = requests.get(
        "http://localhost:8000/long-term-satisfaction", json={"observation": obs}
    )
    response_json = resp.json()
    print(f"<- got {response_json}")
    obs, _, _, _ = offline_rl_env.step(np.array(response_json["action"]))


-> Sending observation OrderedDict([('item', [[0.978618323802948], [0.7991585731506348], [0.4614793658256531], [0.7805292010307312], [0.11827442795038223], [0.6399210095405579], [0.14335328340530396], [0.9446688890457153], [0.5218483209609985], [0.4146619439125061], [0.26455560326576233], [0.7742336988449097], [0.4561503231525421], [0.568433940410614], [0.018789799883961678], [0.6176354885101318], [0.6120957136154175], [0.6169340014457703], [0.9437480568885803], [0.681820273399353]])])


(ServeReplica:ServeModel pid=7532) C:\Users\Paul\AppData\Roaming\Python\Python39\site-packages\gym\spaces\box.py:155: UserWarning: WARN: Casting input x to numpy array.
(ServeReplica:ServeModel pid=7532)   logger.warn("Casting input x to numpy array.")
(ServeReplica:ServeModel pid=7532) INFO 2022-09-05 14:21:56,668 ServeModel ServeModel#HHIaQS replica.py:482 - HANDLE __call__ OK 15.1ms
(HTTPProxyActor pid=22640) INFO 2022-09-05 14:21:56,669 http_proxy 127.0.0.1 http_proxy.py:315 - GET /long-term-satisfaction 200 20.1ms


<- got {'action': 193}
-> Sending observation OrderedDict([('item', [[0.35950788855552673], [0.43703195452690125], [0.6976311802864075], [0.0602254718542099], [0.6667667031288147], [0.670637845993042], [0.21038256585597992], [0.12892629206180573], [0.31542834639549255], [0.36371076107025146], [0.5701967477798462], [0.4386015236377716], [0.9883738160133362], [0.10204481333494186], [0.20887675881385803], [0.16130951046943665], [0.6531082987785339], [0.25329160690307617], [0.4663107693195343], [0.24442559480667114]])])
<- got {'action': 149}
-> Sending observation OrderedDict([('item', [[0.15896958112716675], [0.11037514358758926], [0.6563295722007751], [0.13818295300006866], [0.1965823620557785], [0.3687251806259155], [0.8209932446479797], [0.09710127860307693], [0.8379449248313904], [0.0960984081029892], [0.9764594435691833], [0.4686512053012848], [0.9767611026763916], [0.6048455238342285], [0.7392635941505432], [0.03918779268860817], [0.28280696272850037], [0.12019655853509903], [0.296

(HTTPProxyActor pid=22640) INFO 2022-09-05 14:21:58,714 http_proxy 127.0.0.1 http_proxy.py:315 - GET /long-term-satisfaction 200 3.8ms
(ServeReplica:ServeModel pid=7532) INFO 2022-09-05 14:21:58,713 ServeModel ServeModel#HHIaQS replica.py:482 - HANDLE __call__ OK 1.8ms


<- got {'action': 264}
-> Sending observation OrderedDict([('item', [[0.3179831802845001], [0.414262980222702], [0.06414749473333359], [0.6924721002578735], [0.5666014552116394], [0.26538950204849243], [0.5232480764389038], [0.09394051134586334], [0.5759465098381042], [0.9292961955070496], [0.3185689449310303], [0.6674103736877441], [0.13179786503314972], [0.7163271903991699], [0.28940609097480774], [0.18319135904312134], [0.5865129232406616], [0.02010754682123661], [0.8289400339126587], [0.004695476032793522]])])


(HTTPProxyActor pid=22640) INFO 2022-09-05 14:22:00,740 http_proxy 127.0.0.1 http_proxy.py:315 - GET /long-term-satisfaction 200 4.5ms
(ServeReplica:ServeModel pid=7532) INFO 2022-09-05 14:22:00,739 ServeModel ServeModel#HHIaQS replica.py:482 - HANDLE __call__ OK 1.3ms


<- got {'action': 76}
-> Sending observation OrderedDict([('item', [[0.6778165102005005], [0.2700079679489136], [0.7351940274238586], [0.9621885418891907], [0.2487531453371048], [0.5761573314666748], [0.5920419096946716], [0.5722519159317017], [0.22308163344860077], [0.9527490139007568], [0.4471253752708435], [0.8464086651802063], [0.6994792819023132], [0.2974369525909424], [0.8137978315353394], [0.396505743265152], [0.8811032176017761], [0.5812729001045227], [0.8817353844642639], [0.6925315856933594]])])


(HTTPProxyActor pid=22640) INFO 2022-09-05 14:22:02,785 http_proxy 127.0.0.1 http_proxy.py:315 - GET /long-term-satisfaction 200 4.1ms
(ServeReplica:ServeModel pid=7532) INFO 2022-09-05 14:22:02,784 ServeModel ServeModel#HHIaQS replica.py:482 - HANDLE __call__ OK 1.0ms


<- got {'action': 207}


(HTTPProxyActor pid=22640) INFO 2022-09-05 14:22:04,836 http_proxy 127.0.0.1 http_proxy.py:315 - GET /long-term-satisfaction 200 5.4ms
(ServeReplica:ServeModel pid=7532) INFO 2022-09-05 14:22:04,834 ServeModel ServeModel#HHIaQS replica.py:482 - HANDLE __call__ OK 1.4ms
